In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats)
import time

In [2]:
pbp2022=pd.read_csv(filepath_or_buffer="PBP Data/2021-22_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [3]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2021-22",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season="2021-22",
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [4]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [5]:
hustle_70percent_gp.nlargest(5,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
37,Blake Griffin,1.129787
358,Kyle Lowry,0.622874
523,Taurean Prince,0.563265
419,Moritz Wagner,0.500870
17,Alperen Sengun,0.450000


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [6]:
hustle_70percent_gp.nlargest(5,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
292,John Konchar,1.597633
232,Ja Morant,1.504644
481,Robert Williams III,1.422680
24,Anthony Edwards,1.409577
210,Hamidou Diallo,1.397260


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(5,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
195,Gary Payton II,5.238323
400,Matisse Thybulle,5.112840
128,De'Anthony Melton,4.681545
55,Bruce Brown,4.528507
137,Dejounte Murray,4.313703


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(5,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
489,Rudy Gobert,18.067290
244,Jakob Poeltl,16.212587
163,Drew Eubanks,16.015748
517,Steven Adams,15.820312
322,Jusuf Nurkic,15.463636


# The Deadshot Award (presented by Ray Allen/Reggie Miller)

best qualifying 3 point percentage (Basketball-Reference)

1. P.J. Tucker (46.9%)
2. Joe Harris (46.6%)
3. Eric Gordon (44.8%)
4. Tyrese Haliburton (44.5%)
5. P.J. Washington (44.3%)

# The Stormtrooper Award

worst qualifying 2 point percentage (Basketball-Reference)

1. Ricky Rubio (38%)
2. Devonte' Graham (40%)
3. Buddy Hield (41.2%)
4. Lonzo Ball (41.4%)
5. Nickeil Alexander-Walker & Saddiq Bey (42.2%)

# The No Fly Zone Award (presented by Dikembe Mutumbo)*

most blocked dunks as the blocking player

In [9]:
misses=pbp2022[pbp2022.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp2022.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
33,Daniel Gafford,6
35,Darius Bazley,6
105,Karl-Anthony Towns,6
150,Precious Achiuwa,6
72,Ivica Zubac,5
157,Rudy Gobert,5
178,Tristan Thompson,5


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop\_soup for the idea and asw7412 for the sponsor)

In [10]:
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER3_NAME,count
7,Andre Drummond,8
126,Myles Turner,8
32,Daniel Gafford,7
76,Jaren Jackson Jr.,7
78,Jarrett Allen,7


# The "If He Dies, He Dies" Award (presented by Tom Thibodeau, sponsored by Ivan Drago)

most minutes played per game (Basketball-Reference) (credit to FurryCrew for the idea)

1. Fred VanVleet (37.5)
2. Kevin Durant (37.1)
3. James Harden & Jayson Tatum (36.6)
4. Damian Lillard (36.4)

alternatively: most total minutes played (Basketball-Reference) (credit to FrankEMartindale for the idea)

1. Russell Westbrook (1427)
2. Miles Bridges (1359)
3. Domantas Sabonis (1351)
4. Julius Randle (1350)
5. Mikal Bridges (1325)

# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears, presented by Gary Payton & Shawn Kemp)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [11]:
#get alley oops
alley_oops=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Alley',na=False)]
#remove missed alley oop attempts
made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
#remove alley oops that have missing 2nd player (self-alley oop or perhaps missing data?)
made_alley_oops_complete=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
made_alley_oops_complete['p1']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
made_alley_oops_complete['p2']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
made_alley_oops_complete.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,p1,p2,count
163,Clint Capela,Trae Young,42
195,Darius Garland,Jarrett Allen,28
477,Mike Conley,Rudy Gobert,26
470,Marcus Smart,Robert Williams III,25
408,John Collins,Trae Young,19


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [12]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)
hustle_50percent_gp_ranks.nsmallest(n=5,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
22,Anfernee Simons,0.231198,0.175487,0.030641,0.022284,0.147632,0.607242
100,D.J. Augustin,0.231198,0.008357,0.047354,0.072423,0.303621,0.662953
578,Will Barton,0.231198,0.206128,0.406685,0.013928,0.122563,0.980501
286,Joe Ingles,0.231198,0.069638,0.178273,0.451253,0.072423,1.002786
526,Terrence Ross,0.231198,0.122563,0.506964,0.100279,0.044568,1.005571


In [21]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME']=='Patrick Beverley'].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
456,Patrick Beverley,0.980501,0.052925,0.830084,0.030641,0.849582,2.743733


# The "Mr. Fantastic" Award (presented by Shane Battier, the No-Stats All-Star)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [13]:
hustle_50percent_gp_ranks.nlargest(n=5,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
202,Goga Bitadze,0.894150,0.955432,0.924791,0.945682,1.000000,4.720056
529,Thaddeus Young,0.969359,0.857939,0.896936,0.966574,0.791086,4.481894
162,Draymond Green,0.908078,0.813370,0.832869,0.905292,0.935933,4.395543
309,Josh Okogie,0.935933,0.682451,0.913649,0.708914,0.933148,4.174095
17,Alperen Sengun,0.977716,0.880223,0.935933,0.955432,0.349582,4.098886


# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8), presented by Glen "Big Baby" Davis)*

most charges committed (credit to Kdog122025 for the idea)

In [14]:
charges=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
102,Giannis Antetokounmpo,11
126,Jaren Jackson Jr.,10
133,Jerami Grant,10
148,Julius Randle,10
164,Kyle Kuzma,10


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [15]:
goaltends=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
21,Clint Capela,11
106,Myles Turner,11
58,JaVale McGee,8
126,Robert Williams III,8
0,Aaron Gordon,7
4,Andre Drummond,7


# The "Master Baiter" Award (sponsored by Bass Pro Shops & Kleenex)

most 3-point shooting fouls drawn (PBPStats.com)

1. James Harden (27)
2. Stephen Curry (19)
3. Garrison Mathews (14)
4. Khris Middleton (13)
5. Terrence Ross (11)

# The Anti-Verticality Award (presented by Roy Hibbert)*

most 3-point shooting fouls committed (credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea)

In [16]:
third_free_throws=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp2022['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME_x,count
98,Garrett Temple,6
80,Dillon Brooks,5
144,Josh Richardson,5
261,Tyrese Maxey,5
7,Alec Burks,4
37,Carmelo Anthony,4
43,Chris Duarte,4
76,Desmond Bane,4
111,Isaac Okoro,4
146,Juan Toscano-Anderson,4


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played

1. Luka Doncic (87.7%)
2. Shai Gilgeous-Alexander (86.8%)
3. Chris Paul (84.8%)
4. Trae Young (84.2%)
5. Kevin Pangos (80%)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

1. Juancho Hernangomez (100%)
2. Ben McLemore (96.3%)
3. Mike Muscala & Maxi Kleber (94.9%)
4. Wayne Ellington (94.8%)

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award

Lowest contested rebound percentage, minimum 50% of games played

In [17]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
376,Justin Robinson,0.048
230,Tyus Jones,0.060
322,Kira Lewis Jr.,0.079
100,D.J. Augustin,0.081
111,Bryn Forbes,0.095


In [22]:
reb_w_gp_qualify[reb_w_gp_qualify['PLAYER_NAME']=='Carmelo Anthony'][['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
273,Carmelo Anthony,0.308


alternatively: restricting to players > 6 foot 6 inches in height

In [18]:
player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
46,Furkan Korkmaz,0.107
5,Amir Coffey,0.145
163,Ziaire Williams,0.147
17,Cedi Osman,0.152
107,Maurice Harkless,0.159


# The "Where There's a Will, There's a Way" Award (presented by Dennis Rodman)

Highest contested rebound percentage, minimum 50% of games played

In [19]:
reb_w_gp_qualify.nlargest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
103,Jock Landale,0.681
109,Jakob Poeltl,0.584
81,Moses Brown,0.579
398,Luka Garza,0.579
73,Mitchell Robinson,0.578


alternatively: restricting to players < 6 foot 7 inches in height

In [20]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
110,Justin Champagnie,0.537
144,Moses Moody,0.524
178,Thanasis Antetokounmpo,0.442
128,Leandro Bolmaro,0.400
121,Kenyon Martin Jr.,0.399


# The Rotation Awards

**value depth over one solitary star. used basketball-reference's play-by-play position percentages as initial baseline for position groupings**

# The Best Guard Rotation Award (sponsored by Buckingham Palace)

# The Best Big Rotation Award (jointly sponsored by Tom Hanks, Cadbury and Sex and the City)

# The Best Wing Rotation Award (co-sponsored by Lou Williams and Magic City)